# Report of Permuted MNIST Challenge

In [42]:
# For clearing the old environment before loading the latest one

!rm -rf /content/SD
!rm -rf /content/permuted_mnist

Cloning into 'SD'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 164 (delta 57), reused 112 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (164/164), 12.54 MiB | 10.48 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Cloning into 'permuted_mnist'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 194 (delta 77), reused 163 (delta 50), pack-reused 0 (from 0)
Receiving objects: 100% (194/194), 12.62 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (77/77), done.
✅ 环境已清理并重新导入完毕。


In [43]:
!git clone https://github.com/tiphddddd/SD
!git clone https://github.com/ml-arena/permuted_mnist.git
import sys
sys.path.append('/content/SD')
import sys
sys.path.append('/content/permuted_mnist')

fatal: destination path 'SD' already exists and is not an empty directory.
fatal: destination path 'permuted_mnist' already exists and is not an empty directory.


In [44]:
import numpy as np
import time
import matplotlib.pyplot as plt
from typing import Dict, List
import torch

# Import the environment and agents
from permuted_mnist.env.permuted_mnist import PermutedMNISTEnv

# Create environment with 10 episodes (tasks)
env = PermutedMNISTEnv(number_episodes=10)

# Set seed for reproducibility
env.set_seed(42)

print(f"Environment created with {env.number_episodes} permuted tasks")
print(f"Training set size: {env.train_size} samples")
print(f"Test set size: {env.test_size} samples")

Environment created with 10 permuted tasks
Training set size: 60000 samples
Test set size: 10000 samples


# Ⅰ  Problem Statement

This challenge is based on the **Permuted MNIST** dataset, which represents a typical **fast-adaptation meta-learning** scenario.  
Unlike the traditional MNIST handwritten digit classification task, Permuted MNIST introduces two types of random permutations for each task:

### 1️⃣ Pixel Permutation
The 784 pixels of each input image are randomly shuffled, destroying the original spatial structure. Different tasks use different permutation patterns, so the model cannot rely on convolutional spatial priors.

### 2️⃣ Label Permutation
The digit labels in each task are remapped (for example, all “3”s become “7”s, and all “7”s become “1”s). This means the model must “relearn from scratch” the class-to-label correspondence in every new task.

### ⚙️ System Constraints
- 💻 **Computation**: CPU only (2 cores), 4GB memory.  
- ⏱ **Time constraint**: The total training and inference time per task must be kept within **1 minute**.

Hence, the core goal of this challenge is:  
> **To design a learning algorithm capable of fast convergence and stable generalization under extremely limited computational resources.**


## Ⅱ Methodology

To tackle the dual challenges of **pixel permutation** and **label permutation** in the Permuted MNIST task, this project follows a systematic, **“simple-to-advanced, step-by-step optimization”** strategy.  
The process ultimately results in a balanced and efficient model: the **TorchMLP Agent**.  
Below is a record of the model’s evolution from the baseline to the final optimized version.


### 1️⃣ Baseline Exploration

#### (1) Random and Linear Models
- **Random Predictor**: Used as a zero baseline to verify the environment interface and evaluation pipeline.  
- **Linear Classifier**: A single-layer softmax regression model with L2 regularization.  
  - Performs decently under a fixed permutation (~70%) but generalizes poorly across tasks.

#### (2) Initial MLP Design
- **TorchMLP**: Two fully connected layers `100 → 100 → 10` with ReLU activation.  
  - Training parameters: `epochs=3, batch_size=128`.  
  - This serves as the starting point for later improvements.


### 2️⃣ Feature Engineering (FE)

To reduce brightness and amplitude variations between different permutation tasks and improve robustness, the following lightweight preprocessing is applied:  
- **Pixel Normalization**: If input range is `[0,255]`, rescale to `[0,1]`.  
- **Per-sample L2 Normalization**: Each sample is normalized by its L2 norm to mitigate intensity variation (acts like brightness adjustment).  


### 3️⃣ Improvement of the Model Structure (IMS)

#### (1) Expanded Hidden Dimensions
- The early structure `100→100` is expanded to `256→128` to enhance nonlinear representation capacity.

#### (2) BatchNorm + Activation Enhancement
- Add **Batch Normalization** after each linear layer to stabilize training.  
- Activation combination: use **SiLU** in the first layer (smooth gradients, faster convergence) and **ReLU** in the second (better sparsity).

#### (3) Residual Bottleneck Block
- Introduce a lightweight residual block within the 256-dimensional layer: `256 → 64 → 256`.  
- Structure: `BN → SiLU → Linear → BN → SiLU → Linear`, followed by a pre-activation residual connection `x + h`.

#### (4) Regularization on the Classification Head
- Apply **Weight Normalization** to the final layer `Linear(128→10)` to reduce scale instability.  
- Disable it before quantization for compatibility.

> In addition, **Label Smoothing (ε=0.05)** is introduced to prevent overconfidence on single labels and improve generalization across label-permuted tasks.


### 4️⃣ Optimizer (OP)

#### Comparison and Selection
- Compared **Adam**, **SGD**, and **RMSprop**:
  - Adam: fast convergence but unstable oscillations;  
  - SGD: stable but slow;  
  - ✅ **RMSprop**: balanced speed and smoothness on CPU — selected as the final optimizer. Adding a **Cosine Annealing scheduler** further smooths the loss curve and stabilizes validation accuracy.


### 5️⃣ Model Compression (MC)

To further improve inference speed and efficiency under CPU constraints, two lightweight compression techniques were tested:  
- **Dynamic INT8 Quantization**: Apply dynamic quantization to all linear layers, achieving significant acceleration with negligible accuracy loss.  
- **Prune40% + INT8**: Perform 40% pruning before quantization — faster inference but with slight accuracy degradation.  

The final deployment uses **Dynamic INT8 Quantization**, which preserves stable accuracy while greatly accelerating inference.


### 6️⃣ Hyperparameter Tuning and Final Configuration

A grid search was conducted over:  
`epochs ∈ {5, 7, 10, 15}`, `batch_size ∈ {100, 128}`  

Final choice:  
> **epochs = 10**, **batch_size = 128**

This configuration achieves over **98.5% accuracy**  
while keeping total runtime per task within **35–38 seconds (CPU mode)**.


# Ⅱ.From Baselines to the Final Model Evolution Path

### 1️⃣ Baseline Exploration

#### (1) Random and Linear Models
- **Random Predictor**: Serves as a zero baseline to verify the environment interface and evaluation process;


In [9]:
from SD.models.random import Agent as RandomAgent

# Reset environment for fresh start
env.reset()
env.set_seed(42)

# Create random agent
random_agent = RandomAgent(output_dim=10, seed=42)

# Track performance
random_accuracies = []
random_times = []

print("Evaluating Random Agent (Baseline)")
print("="*50)

# Evaluate on all tasks
task_num = 1
while True:
    task = env.get_next_task()
    if task is None:
        break

    start_time = time.time()
    random_agent.train(task['X_train'], task['y_train'])
    predictions = random_agent.predict(task['X_test'])
    elapsed_time = time.time() - start_time
    accuracy = env.evaluate(predictions, task['y_test'])

    random_accuracies.append(accuracy)
    random_times.append(elapsed_time)

    print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.4f}s")
    task_num += 1

print(f"\nRandom Agent Summary:")
print(f"  Mean accuracy: {np.mean(random_accuracies):.2%} ± {np.std(random_accuracies):.2%}")
print(f"  Total time: {np.sum(random_times):.2f}s")


Evaluating Random Agent (Baseline)
Task 1: Accuracy = 9.96%, Time = 0.0003s
Task 2: Accuracy = 9.70%, Time = 0.0003s
Task 3: Accuracy = 10.41%, Time = 0.0004s
Task 4: Accuracy = 10.02%, Time = 0.0005s
Task 5: Accuracy = 10.23%, Time = 0.0004s
Task 6: Accuracy = 9.94%, Time = 0.0003s
Task 7: Accuracy = 10.29%, Time = 0.0003s
Task 8: Accuracy = 10.27%, Time = 0.0003s
Task 9: Accuracy = 9.93%, Time = 0.0003s
Task 10: Accuracy = 10.09%, Time = 0.0003s

Random Agent Summary:
  Mean accuracy: 10.08% ± 0.20%
  Total time: 0.00s


- **Linear Classifier (Logistic Regression)**: A single-layer softmax regression model with L2 regularization;  
  - Performs reasonably well under a fixed permutation (≈90%) but generalizes poorly across different tasks.


In [11]:
from SD.models.linear import Agent as LinearAgent

# Reset environment
env.reset()
env.set_seed(42)

# Create linear agent
linear_agent = LinearAgent(input_dim=784, output_dim=10, learning_rate=0.01)

# Track performance
linear_accuracies = []
linear_times = []

print("Evaluating Linear Agent")
print("="*50)

# Evaluate on all tasks
task_num = 1
while True:
    task = env.get_next_task()
    if task is None:
        break
    linear_agent.reset()
    start_time = time.time()
    linear_agent.train(task['X_train'], task['y_train'], epochs=5, batch_size=32)
    predictions = linear_agent.predict(task['X_test'])
    elapsed_time = time.time() - start_time
    accuracy = env.evaluate(predictions, task['y_test'])

    linear_accuracies.append(accuracy)
    linear_times.append(elapsed_time)

    print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.2f}s")
    task_num += 1

print(f"\nLinear Agent Summary:")
print(f"  Mean accuracy: {np.mean(linear_accuracies):.2%} ± {np.std(linear_accuracies):.2%}")
print(f"  Total time: {np.sum(linear_times):.2f}s")

Evaluating Linear Agent
Task 1: Accuracy = 90.77%, Time = 2.98s
Task 2: Accuracy = 90.68%, Time = 4.04s
Task 3: Accuracy = 90.73%, Time = 3.15s
Task 4: Accuracy = 90.79%, Time = 3.93s
Task 5: Accuracy = 90.62%, Time = 2.92s
Task 6: Accuracy = 90.92%, Time = 3.39s
Task 7: Accuracy = 90.87%, Time = 2.96s
Task 8: Accuracy = 90.79%, Time = 2.98s
Task 9: Accuracy = 90.70%, Time = 3.69s
Task 10: Accuracy = 90.84%, Time = 2.92s

Linear Agent Summary:
  Mean accuracy: 90.77% ± 0.09%
  Total time: 32.98s


#### (2) Initial MLP Design
- **TorchMLP**: Building upon the LinearAgent, we aimed for a stronger starting point.  
  A two-layer fully connected network `100 → 100 → 10` with ReLU activation was implemented,  
  trained with parameters `epochs=3, batch_size=128`.  
  This lightweight TorchMLP achieved promising results and served as the foundation for subsequent optimizations.


In [17]:
from SD.models.torchmlp import Agent as TorchMLP

# Reset environment
env.reset()
env.set_seed(42)

# Create MLP agent
mlp_agent = TorchMLP(
    output_dim=10,
    seed=42,
    hidden_sizes=[100, 100],
    n_epochs=3,
    batch_size=128
)

mlp_agent.verbose = False

# Track performance
mlp_accuracies = []
mlp_times = []

print("Evaluating TorchMLP Agent (100→100→10, epochs=3, batch=128)")
print("=" * 50)

# Evaluate on all tasks
task_num = 1
while True:
    task = env.get_next_task()
    if task is None:
        break

    mlp_agent.reset()
    start_time = time.time()
    mlp_agent.verbose = False
    mlp_agent.train(task['X_train'], task['y_train'])
    predictions = mlp_agent.predict(task['X_test'])
    elapsed_time = time.time() - start_time
    accuracy = env.evaluate(predictions, task['y_test'])

    mlp_accuracies.append(accuracy)
    mlp_times.append(elapsed_time)

    print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.2f}s")
    task_num += 1

print(f"\nTorchMLP Agent Summary:")
print(f"  Mean accuracy: {np.mean(mlp_accuracies):.2%} ± {np.std(mlp_accuracies):.2%}")
print(f"  Total time: {np.sum(mlp_times):.2f}s")


Evaluating TorchMLP Agent (100→100→10, epochs=3, batch=128)
Task 1: Accuracy = 97.31%, Time = 5.80s
Task 2: Accuracy = 97.40%, Time = 4.90s
Task 3: Accuracy = 97.55%, Time = 5.18s
Task 4: Accuracy = 97.13%, Time = 5.87s
Task 5: Accuracy = 97.47%, Time = 6.03s
Task 6: Accuracy = 97.45%, Time = 6.72s
Task 7: Accuracy = 97.45%, Time = 5.50s
Task 8: Accuracy = 97.44%, Time = 5.80s
Task 9: Accuracy = 97.37%, Time = 7.05s
Task 10: Accuracy = 97.34%, Time = 6.00s

TorchMLP Agent Summary:
  Mean accuracy: 97.39% ± 0.11%
  Total time: 58.87s


### 2️⃣ Feature Engineering (FE)

To reduce brightness and amplitude variations between different permutation tasks and improve robustness, the following lightweight preprocessing is applied:  
- **Pixel Normalization**: If input range is `[0,255]`, rescale to `[0,1]`.  
- **Per-sample L2 Normalization**: Each sample is normalized by its L2 norm to mitigate intensity variation (acts like brightness adjustment).  



In [24]:
from SD.utils.data import _as_float_01,_l2_per_sample

def _to_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return x

def _fe_for_agent(x):
    x01 = _as_float_01(x)
    xl2 = _l2_per_sample(x01)
    xl2 = _to_numpy(xl2)
    return (xl2 * 255.0).astype(np.float32)

# Reset environment
env.reset()
env.set_seed(42)

# Create MLP agent
mlp_agent = TorchMLP(
    output_dim=10,
    seed=42,
    hidden_sizes=[100, 100],
    n_epochs=3,
    batch_size=128
)
mlp_agent.verbose = False

# Track performance
mlp_accuracies = []
mlp_times = []

print("Evaluating TorchMLP Agent + FE (100→100→10, epochs=3, batch=128)")
print("=" * 50)

# Evaluate on all tasks
task_num = 1
while True:
    task = env.get_next_task()
    if task is None:
        break

    # ---- Apply FE ----
    Xtr = _fe_for_agent(task['X_train'])
    Xte = _fe_for_agent(task['X_test'])

    mlp_agent.reset()
    start_time = time.time()
    mlp_agent.verbose = False
    mlp_agent.train(Xtr, task['y_train'])
    predictions = mlp_agent.predict(Xte)
    elapsed_time = time.time() - start_time
    accuracy = env.evaluate(predictions, task['y_test'])

    mlp_accuracies.append(accuracy)
    mlp_times.append(elapsed_time)

    print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.2f}s")
    task_num += 1

print(f"\nTorchMLP Agent + FE Summary:")
print(f"  Mean accuracy: {np.mean(mlp_accuracies):.2%} ± {np.std(mlp_accuracies):.2%}")
print(f"  Total time: {np.sum(mlp_times):.2f}s")

Evaluating TorchMLP Agent + FE (100→100→10, epochs=3, batch=128)
Task 1: Accuracy = 97.41%, Time = 5.66s
Task 2: Accuracy = 97.41%, Time = 4.78s
Task 3: Accuracy = 97.55%, Time = 4.72s
Task 4: Accuracy = 97.40%, Time = 5.00s
Task 5: Accuracy = 97.49%, Time = 5.58s
Task 6: Accuracy = 97.86%, Time = 4.92s
Task 7: Accuracy = 97.64%, Time = 4.73s
Task 8: Accuracy = 97.35%, Time = 4.76s
Task 9: Accuracy = 97.30%, Time = 6.05s
Task 10: Accuracy = 97.48%, Time = 5.83s

TorchMLP Agent + FE Summary:
  Mean accuracy: 97.49% ± 0.15%
  Total time: 52.05s


### 3️⃣ Improvement of the Model Structure (IMS)

#### (1) Expanded Hidden Dimensions
- The early structure `100→100` is expanded to `256→128` to enhance nonlinear representation capacity.

#### (2) BatchNorm + Activation Enhancement
- Add **Batch Normalization** after each linear layer to stabilize training.  
- Activation combination: use **SiLU** in the first layer (smooth gradients, faster convergence) and **ReLU** in the second (better sparsity).

#### (3) Residual Bottleneck Block
- Introduce a lightweight residual block within the 256-dimensional layer: `256 → 64 → 256`.  
- Structure: `BN → SiLU → Linear → BN → SiLU → Linear`, followed by a pre-activation residual connection `x + h`.

#### (4) Regularization on the Classification Head
- Apply **Weight Normalization** to the final layer `Linear(128→10)` to reduce scale instability.  
- Disable it before quantization for compatibility.

> In addition, **Label Smoothing (ε=0.05)** is introduced to prevent overconfidence on single labels and improve generalization across label-permuted tasks.

In [28]:
from SD.utils.model import _MLP_ResBN,Bottleneck  # SiLU+BN / ReLU / Bottleneck / WeightNorm
from SD.utils.loss import SmoothCE #LableSmoothing

def _train_one_task(model, X_tr_np, y_tr_np, *, epochs=3, batch_size=128, lr=1e-3):
    Xtr = torch.from_numpy(X_tr_np).float() / 255.0
    ytr = torch.from_numpy(np.asarray(y_tr_np).reshape(-1)).long()
    ds = torch.utils.data.TensorDataset(Xtr, ytr)
    loader = torch.utils.data.DataLoader(
        ds, batch_size=batch_size, shuffle=True, drop_last=False,
        num_workers=0, pin_memory=False)
    opt  = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, momentum=0.0, centered=False, weight_decay=0.0)
    crit = SmoothCE(eps=0.05, num_classes=10)
    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            opt.zero_grad(set_to_none=True)
            logits = model(xb.view(xb.size(0), -1))
            loss = crit(logits, yb)
            loss.backward()
            opt.step()
@torch.no_grad()

def _predict(model, X_te_np):
    Xte = torch.from_numpy(X_te_np).float() / 255.0
    model.eval()
    bs = 4096
    outs = []
    for i in range(0, Xte.shape[0], bs):
        logits = model(Xte[i:i+bs].view(-1, 28*28))
        outs.append(torch.argmax(logits, dim=1).cpu().numpy())
    return np.concatenate(outs, axis=0)
env.reset()
env.set_seed(42)

accs, times = [], []

print("Evaluating TorchMLP + FE + IMS (256→128, BN+SiLU/ReLU, Bottleneck, WN, LS ε=0.05)  (epochs=3, batch=128)")
print("=" * 80)

task_id = 1
while True:
    task = env.get_next_task()
    if task is None:
        break

    # ---- Apply Feature Engineering (reuse previous FE) ----
    Xtr = _fe_for_agent(task['X_train'])
    Xte = _fe_for_agent(task['X_test'])

    # ---- Build improved model ----
    model = _MLP_ResBN(in_dim=784, out_dim=10)

    # ---- Train (reuse existing training helper, just change loss) ----
    t0 = time.time()
    _train_one_task(model, Xtr, task['y_train'], epochs=3, batch_size=128, lr=1e-3)
    preds = _predict(model, Xte)
    elapsed = time.time() - t0

    acc = env.evaluate(preds, task['y_test'])
    accs.append(acc)
    times.append(elapsed)

    print(f"Task {task_id}: Accuracy = {acc:.2%}, Time = {elapsed:.2f}s")
    task_id += 1

print("\nTorchMLP + FE + IMS Summary:")
print(f"  Mean accuracy: {np.mean(accs):.2%} ± {np.std(accs):.2%}")
print(f"  Total time: {np.sum(times):.2f}s")

Evaluating TorchMLP + FE + IMS (256→128, BN+SiLU/ReLU, Bottleneck, WN, LS ε=0.05)  (epochs=3, batch=128)
Task 1: Accuracy = 98.23%, Time = 15.88s
Task 2: Accuracy = 98.31%, Time = 13.94s
Task 3: Accuracy = 98.19%, Time = 14.84s
Task 4: Accuracy = 98.05%, Time = 14.15s
Task 5: Accuracy = 98.07%, Time = 14.96s
Task 6: Accuracy = 98.07%, Time = 14.13s
Task 7: Accuracy = 98.28%, Time = 14.44s
Task 8: Accuracy = 98.06%, Time = 13.79s
Task 9: Accuracy = 98.07%, Time = 13.88s
Task 10: Accuracy = 98.13%, Time = 13.91s

TorchMLP + FE + IMS Summary:
  Mean accuracy: 98.15% ± 0.09%
  Total time: 143.91s


### 4️⃣ Optimizer (OP)

#### Comparison and Selection
- Compared **Adam**, **SGD**, and **RMSprop**:
  - Adam: fast convergence but unstable oscillations;  
  - SGD: stable but slow;  
  - ✅ **RMSprop**: balanced speed and smoothness on CPU — selected as the final optimizer. Adding a **Cosine Annealing scheduler** further smooths the loss curve and stabilizes validation accuracy.


In [50]:
import pandas as pd
from SD.utils.warmcosine import WarmupCosine

OPT_FNS = {
    "Adam":    lambda p: torch.optim.Adam(p, lr=1e-3, betas=(0.9, 0.999), weight_decay=0.0),
    "SGD":     lambda p: torch.optim.SGD(p, lr=1e-2, momentum=0.9, nesterov=False, weight_decay=0.0),
    "RMSprop": lambda p: torch.optim.RMSprop(p, lr=1e-3, alpha=0.99, momentum=0.0, centered=False, weight_decay=0.0),
}

EPOCHS = 3
BATCH  = 128

def run_with_optimizer(opt_name, opt_fn):
    env.reset()
    env.set_seed(42)

    accs, times = [], []

    print(f"\nEvaluating TorchMLP + FE + IMS  (OPT={opt_name}, epochs={EPOCHS}, batch={BATCH})")
    print("=" * 90)

    task_id = 1
    while True:
        task = env.get_next_task()
        if task is None:
            break

        # ---- Feature Engineering (reuse) ----
        Xtr = _fe_for_agent(task['X_train'])
        Xte = _fe_for_agent(task['X_test'])

        # ---- Build model (reuse IMS) ----
        model = _MLP_ResBN(in_dim=784, out_dim=10)
        crit  = SmoothCE(eps=0.05, num_classes=10)

        # ---- Dataloader (same as step 3) ----
        Xtr_t = torch.from_numpy(Xtr).float() / 255.0
        ytr_t = torch.from_numpy(np.asarray(task['y_train']).reshape(-1)).long()
        ds    = torch.utils.data.TensorDataset(Xtr_t, ytr_t)
        loader= torch.utils.data.DataLoader(ds, batch_size=BATCH, shuffle=True, drop_last=False,
                                            num_workers=0, pin_memory=False)

        # ---- Train with chosen optimizer ----
        opt = opt_fn(model.parameters())
        sch = None
        if opt_name == "RMSprop":
            sch = WarmupCosine(opt, total_epochs=EPOCHS, warmup_epochs=1)

        t0  = time.time()
        model.train()
        for _ in range(EPOCHS):
            for xb, yb in loader:
                opt.zero_grad(set_to_none=True)
                logits = model(xb.view(xb.size(0), -1))
                loss   = crit(logits, yb)
                loss.backward()
                opt.step()
            if sch is not None:
                sch.step()

        # ---- Predict (same as step 3) ----
        @torch.no_grad()
        def _predict_batch(X_te_np):
            Xte_t = torch.from_numpy(X_te_np).float() / 255.0
            model.eval()
            bs = 4096
            outs = []
            for i in range(0, Xte_t.shape[0], bs):
                logits = model(Xte_t[i:i+bs].view(-1, 28*28))
                outs.append(torch.argmax(logits, dim=1).cpu().numpy())
            return np.concatenate(outs, axis=0)

        preds   = _predict_batch(Xte)
        elapsed = time.time() - t0

        acc = env.evaluate(preds, task['y_test'])
        accs.append(acc)
        times.append(elapsed)

        print(f"Task {task_id}: Accuracy = {acc:.2%}, Time = {elapsed:.2f}s")
        task_id += 1

    print(f"\n{opt_name} Summary:")
    print(f"  Mean accuracy: {np.mean(accs):.2%} ± {np.std(accs):.2%}")
    print(f"  Total time: {np.sum(times):.2f}s")

    return {
        "optimizer": opt_name,
        "mean_acc":  float(np.mean(accs)),
        "std_acc":   float(np.std(accs)),
        "total_time":float(np.sum(times)),
        "n_tasks":   len(accs),
    }

# -------- Run all three and tabulate --------
results = []
for name, fn in OPT_FNS.items():
    res = run_with_optimizer(name, fn)
    results.append(res)

df = pd.DataFrame(results)
df = df[["optimizer", "n_tasks", "mean_acc", "std_acc", "total_time"]]
df["mean_acc(%)"] = (df["mean_acc"] * 100).round(2)
df["std_acc(%)"]  = (df["std_acc"]  * 100).round(2)
df["total_time(s)"] = df["total_time"].round(2)
df = df.drop(columns=["mean_acc", "std_acc", "total_time"])

print("\n=== Optimizer Comparison (Step 4) ===")
print(df.to_string(index=False))



Evaluating TorchMLP + FE + IMS  (OPT=Adam, epochs=3, batch=128)


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Task 1: Accuracy = 98.12%, Time = 15.04s
Task 2: Accuracy = 98.28%, Time = 15.28s
Task 3: Accuracy = 98.23%, Time = 17.24s
Task 4: Accuracy = 98.12%, Time = 14.86s
Task 5: Accuracy = 98.02%, Time = 14.65s
Task 6: Accuracy = 98.03%, Time = 14.79s
Task 7: Accuracy = 98.02%, Time = 14.83s
Task 8: Accuracy = 98.18%, Time = 14.77s
Task 9: Accuracy = 98.35%, Time = 14.88s
Task 10: Accuracy = 98.21%, Time = 14.84s

Adam Summary:
  Mean accuracy: 98.16% ± 0.11%
  Total time: 151.19s

Evaluating TorchMLP + FE + IMS  (OPT=SGD, epochs=3, batch=128)
Task 1: Accuracy = 97.62%, Time = 13.19s
Task 2: Accuracy = 97.44%, Time = 12.86s
Task 3: Accuracy = 97.41%, Time = 12.83s
Task 4: Accuracy = 97.26%, Time = 12.64s
Task 5: Accuracy = 97.78%, Time = 13.32s
Task 6: Accuracy = 97.59%, Time = 12.94s
Task 7: Accuracy = 97.39%, Time = 13.83s
Task 8: Accuracy = 97.57%, Time = 13.58s
Task 9: Accuracy = 97.47%, Time = 13.46s
Task 10: Accuracy = 97.44%, Time = 14.41s

SGD Summary:
  Mean accuracy: 97.50% ± 0.14%

### 5️⃣ Model Compression (MC)

During the inference stage, to further improve speed and efficiency under CPU constraints, two lightweight compression techniques were tested:  
- **Dynamic INT8 Quantization**: Applied dynamic quantization to all fully connected layers, significantly boosting inference speed with almost no loss in accuracy;  
- **Prune40% + INT8**: Performed 40% pruning before quantization, achieving faster inference but with a some accuracy drop.  

**Dynamic INT8** worked remarkably well on our model!  
We ultimately adopted it as the deployment scheme, achieving substantial acceleration in inference while maintaining stable accuracy.  
Under certain heavier parameter settings, Dynamic INT8 even reduced inference time by more than tenfold without any loss in precision.  
I believe this was the key factor that allowed our model to maintain competitive accuracy while running significantly faster than other approaches.


In [53]:
def _strip_weight_norms(model):
    for m in model.modules():
        try:
            nn.utils.remove_weight_norm(m, name='weight')
        except Exception:
            pass

def _train_rmsprop_warmcosine(model, Xtr_np, ytr_np, *, epochs=3, batch_size=128, lr=1e-3):
    Xtr_t = torch.from_numpy(Xtr_np).float() / 255.0
    ytr_t = torch.from_numpy(np.asarray(ytr_np).reshape(-1)).long()
    ds    = torch.utils.data.TensorDataset(Xtr_t, ytr_t)
    loader= torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=False,
                                        num_workers=0, pin_memory=False)
    opt   = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, momentum=0.0, centered=False, weight_decay=0.0)
    sched = WarmupCosine(opt, total_epochs=epochs, warmup_epochs=1)
    crit  = SmoothCE(eps=0.05, num_classes=10)

    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            opt.zero_grad(set_to_none=True)
            logits = model(xb.view(xb.size(0), -1))
            loss   = crit(logits, yb)
            loss.backward()
            opt.step()
        sched.step()

def _run_rmsprop_with_compressor(label, compress_mode, *, epochs=3, batch_size=128, lr=1e-3):

    env.reset()
    env.set_seed(42)

    print(f"\nEvaluating [{label}]  RMSprop+WarmCosine + FE + IMS  (epochs={epochs}, batch={batch_size})")
    print("=" * 96)

    accs, times = [], []
    task_id = 1
    while True:
        task = env.get_next_task()
        if task is None:
            break

        # ---- FE（与第三步一致）----
        Xtr = _fe_for_agent(task['X_train'])
        Xte = _fe_for_agent(task['X_test'])

        # ---- build the model ----
        model = _MLP_ResBN(in_dim=784, out_dim=10)

        # ---- train ----
        t0 = time.time()
        _train_rmsprop_warmcosine(model, Xtr, task['y_train'], epochs=epochs, batch_size=batch_size, lr=lr)

        # ---- compressor ----
        if compress_mode == 'int8':
            # remove WN
            _strip_weight_norms(model)
            try:
                model_eval = apply_dynamic_int8_quantization(model)
            except Exception:
                model_eval = torch.ao.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8, inplace=False)

        elif compress_mode == 'prune_int8':
            _strip_weight_norms(model)
            # in_place=True avoid deepcopy trigger the restriction of WN
            model_q, _rep = prune40_int8(model, amount=0.40, exclude_head=True, in_place=True)
            model_eval = model_q
        else:
            raise ValueError(f"Unknown compress_mode: {compress_mode}")

        preds   = _predict(model_eval, Xte)
        elapsed = time.time() - t0

        accs.append(env.evaluate(preds, task['y_test']))
        times.append(elapsed)
        print(f"Task {task_id}: Accuracy = {accs[-1]:.2%}, Time = {elapsed:.2f}s")
        task_id += 1

    mean_acc = float(np.mean(accs))
    std_acc  = float(np.std(accs))
    total_t  = float(np.sum(times))

    print(f"\n[{label}] Summary:")
    print(f"  Mean accuracy: {mean_acc:.2%} ± {std_acc:.2%}")
    print(f"  Total time: {total_t:.2f}s")

    return {
        "variant": label,
        "mean_acc":  mean_acc,
        "std_acc":   std_acc,
        "total_time": total_t,
        "n_tasks":   len(accs),
    }

# ------------------ run the model with 2 compressor ------------------
res_int8       = _run_rmsprop_with_compressor("Dynamic INT8",    "int8",       epochs=3, batch_size=128, lr=1e-3)
res_prune_int8 = _run_rmsprop_with_compressor("Prune40% + INT8", "prune_int8", epochs=3, batch_size=128, lr=1e-3)

# ------------------ comparison table ------------------
results = [res_int8, res_prune_int8]
df = pd.DataFrame(results)
df["mean_acc(%)"]   = (df["mean_acc"] * 100).round(2)
df["std_acc(%)"]    = (df["std_acc"]  * 100).round(2)
df["total_time(s)"] = df["total_time"].round(2)
df = df[["variant", "n_tasks", "mean_acc(%)", "std_acc(%)", "total_time(s)"]]

print("\n=== RMSprop+WarmCosine with Compressors — Comparison ===")
print(df.to_string(index=False))



Evaluating [Dynamic INT8]  RMSprop+WarmCosine + FE + IMS  (epochs=3, batch=128)
Task 1: Accuracy = 98.45%, Time = 15.27s
Task 2: Accuracy = 98.31%, Time = 14.38s
Task 3: Accuracy = 98.26%, Time = 16.50s
Task 4: Accuracy = 98.21%, Time = 15.19s
Task 5: Accuracy = 98.26%, Time = 17.70s
Task 6: Accuracy = 98.43%, Time = 17.06s
Task 7: Accuracy = 98.38%, Time = 18.77s
Task 8: Accuracy = 98.49%, Time = 16.38s
Task 9: Accuracy = 98.30%, Time = 15.90s
Task 10: Accuracy = 98.17%, Time = 15.09s

[Dynamic INT8] Summary:
  Mean accuracy: 98.33% ± 0.10%
  Total time: 162.24s

Evaluating [Prune40% + INT8]  RMSprop+WarmCosine + FE + IMS  (epochs=3, batch=128)
Task 1: Accuracy = 98.09%, Time = 15.16s
Task 2: Accuracy = 98.09%, Time = 16.44s
Task 3: Accuracy = 97.93%, Time = 14.53s
Task 4: Accuracy = 98.18%, Time = 16.25s
Task 5: Accuracy = 98.23%, Time = 14.27s
Task 6: Accuracy = 97.99%, Time = 16.15s
Task 7: Accuracy = 97.91%, Time = 14.50s
Task 8: Accuracy = 97.99%, Time = 15.70s
Task 9: Accuracy 


### 6️⃣ Hyperparameter Tuning and Final Configuration

A grid search was conducted over:  
`epochs ∈ {5, 7, 10, 15}`, `batch_size ∈ {100, 128}`  


In [54]:
def _strip_weight_norms(model):
    for m in model.modules():
        try:
            nn.utils.remove_weight_norm(m, name='weight')
        except Exception:
            pass

def _train_rmsprop_warmcosine(model, Xtr_np, ytr_np, *, epochs=3, batch_size=128, lr=1e-3):
    Xtr_t = torch.from_numpy(Xtr_np).float() / 255.0
    ytr_t = torch.from_numpy(np.asarray(ytr_np).reshape(-1)).long()
    ds    = torch.utils.data.TensorDataset(Xtr_t, ytr_t)
    loader= torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=False,
                                        num_workers=0, pin_memory=False)
    opt   = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, momentum=0.0, centered=False, weight_decay=0.0)
    sched = WarmupCosine(opt, total_epochs=epochs, warmup_epochs=1)
    crit  = SmoothCE(eps=0.05, num_classes=10)

    model.train()
    for _ in range(epochs):
        for xb, yb in loader:
            opt.zero_grad(set_to_none=True)
            logits = model(xb.view(xb.size(0), -1))
            loss   = crit(logits, yb)
            loss.backward()
            opt.step()
        sched.step()

def _to_int8(model):
    _strip_weight_norms(model)
    try:
        return apply_dynamic_int8_quantization(model)
    except Exception:
        return torch.ao.quantization.quantize_dynamic(
            model, {nn.Linear}, dtype=torch.qint8, inplace=False
        )

def _run_config(epochs, batch_size, lr=1e-3):
    env.reset()
    env.set_seed(42)

    print(f"\nEvaluating (epochs={epochs}, batch={batch_size})  RMSprop+WarmCosine + FE + IMS + Dynamic INT8")
    print("=" * 96)

    accs, times = [], []
    tid = 1
    while True:
        task = env.get_next_task()
        if task is None:
            break

        # FE
        Xtr = _fe_for_agent(task['X_train'])
        Xte = _fe_for_agent(task['X_test'])

        # model
        model = _MLP_ResBN(in_dim=784, out_dim=10)

        # train
        t0 = time.time()
        _train_rmsprop_warmcosine(model, Xtr, task['y_train'], epochs=epochs, batch_size=batch_size, lr=lr)

        # compress: dynamic INT8
        model_q = _to_int8(model)

        # predict
        preds   = _predict(model_q, Xte)
        elapsed = time.time() - t0

        acc = env.evaluate(preds, task['y_test'])
        accs.append(acc); times.append(elapsed)
        print(f"Task {tid}: Accuracy = {acc:.2%}, Time = {elapsed:.2f}s")
        tid += 1

    mean_acc = float(np.mean(accs))
    std_acc  = float(np.std(accs))
    total_t  = float(np.sum(times))

    print(f"\n(epochs={epochs}, batch={batch_size}) Summary:")
    print(f"  Mean accuracy: {mean_acc:.2%} ± {std_acc:.2%}")
    print(f"  Total time: {total_t:.2f}s")

    return {
        "epochs": epochs,
        "batch_size": batch_size,
        "n_tasks": len(accs),
        "mean_acc":  mean_acc,
        "std_acc":   std_acc,
        "total_time": total_t,
    }

# --------- run 8 group ---------
grid_epochs = [5, 7, 10, 15]
grid_batch  = [100, 128]

all_results = []
for ep in grid_epochs:
    for bs in grid_batch:
        all_results.append(_run_config(ep, bs, lr=1e-3))

# --------- comparison table ---------
df = pd.DataFrame(all_results)
df["mean_acc(%)"]   = (df["mean_acc"] * 100).round(2)
df["std_acc(%)"]    = (df["std_acc"]  * 100).round(2)
df["total_time(s)"] = df["total_time"].round(2)
df = df[["epochs", "batch_size", "n_tasks", "mean_acc(%)", "std_acc(%)", "total_time(s)"]]
df = df.sort_values(by=["mean_acc(%)","total_time(s)"], ascending=[False, True])

print("\n=== Grid Search: RMSprop+WarmCosine + Dynamic INT8 ===")
print(df.to_string(index=False))



Evaluating (epochs=5, batch=100)  RMSprop+WarmCosine + FE + IMS + Dynamic INT8


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Task 1: Accuracy = 98.49%, Time = 26.75s
Task 2: Accuracy = 98.59%, Time = 27.79s
Task 3: Accuracy = 98.64%, Time = 26.55s
Task 4: Accuracy = 98.54%, Time = 27.07s
Task 5: Accuracy = 98.62%, Time = 26.66s
Task 6: Accuracy = 98.56%, Time = 26.82s
Task 7: Accuracy = 98.54%, Time = 29.11s
Task 8: Accuracy = 98.55%, Time = 27.25s
Task 9: Accuracy = 98.56%, Time = 26.51s
Task 10: Accuracy = 98.60%, Time = 26.55s

(epochs=5, batch=100) Summary:
  Mean accuracy: 98.57% ± 0.04%
  Total time: 271.06s

Evaluating (epochs=5, batch=128)  RMSprop+WarmCosine + FE + IMS + Dynamic INT8
Task 1: Accuracy = 98.50%, Time = 23.97s
Task 2: Accuracy = 98.55%, Time = 23.80s
Task 3: Accuracy = 98.56%, Time = 25.60s
Task 4: Accuracy = 98.62%, Time = 23.95s
Task 5: Accuracy = 98.55%, Time = 26.16s
Task 6: Accuracy = 98.65%, Time = 27.07s
Task 7: Accuracy = 98.59%, Time = 26.98s
Task 8: Accuracy = 98.58%, Time = 24.82s
Task 9: Accuracy = 98.48%, Time = 23.02s
Task 10: Accuracy = 98.56%, Time = 24.30s

(epochs=5, 

Among the eight experimental configurations, **`epochs=10, batch_size=128`** achieved the best balance between accuracy and runtime.  
It reached a **mean accuracy of 98.68%**, one of the highest across all tests, while keeping the **average task time around 51 seconds**, safely below the 60-second evaluation limit.  
Compared with higher epochs (e.g., 15), this setup maintained nearly identical accuracy but with significantly shorter runtime;  
and compared with lower epochs (5 or 7), it achieved more stable and higher convergence with only a modest increase in time.  
Therefore, **`epochs=10, batch_size=128`** was selected as the final configuration for submission.

Moreover, the model actually performs much better online than it does here, it cost way less time than here.

# Ⅲ Reproduction of Best Submission

In [49]:
from SD.agent import TorchMLP as BestAgent

# Reset environment for fresh start
env.reset()
env.set_seed(42)

best_agent = BestAgent(
    output_dim=10,
    seed=42,
    epochs=10,
    batch_size=128,
    lr=1e-3,
    val_ratio=0.2
)

best_accuracies = []
best_times = []

print("Evaluating Best Agent: TorchMLP + FE + SiLU + Bottleneck + WeightNorm + LS + WarmupCosine + Dynamic INT8")
print("=" * 90)

task_num = 1
while True:
    task = env.get_next_task()
    if task is None:
        break

       best_agent.reset()

    start_time = time.time()
    best_agent.train(task['X_train'], task['y_train'])

    best_agent.compress_dynamic_int8()

    predictions = best_agent.predict(task['X_test'])
    elapsed_time = time.time() - start_time

    accuracy = env.evaluate(predictions, task['y_test'])
    best_accuracies.append(accuracy)
    best_times.append(elapsed_time)

    print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.2f}s")
    task_num += 1

print("\nBest Agent Summary:")
print(f"  Mean accuracy: {np.mean(best_accuracies):.2%} ± {np.std(best_accuracies):.2%}")
print(f"  Total time: {np.sum(best_times):.2f}s")


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Evaluating Best Agent: TorchMLP + FE + SiLU + Bottleneck + WeightNorm + LS + WarmupCosine + Dynamic INT8
Task 1: Accuracy = 98.55%, Time = 40.67s
Task 2: Accuracy = 98.57%, Time = 38.33s
Task 3: Accuracy = 98.51%, Time = 38.23s
Task 4: Accuracy = 98.51%, Time = 38.44s
Task 5: Accuracy = 98.50%, Time = 37.73s
Task 6: Accuracy = 98.57%, Time = 38.74s
Task 7: Accuracy = 98.39%, Time = 44.51s
Task 8: Accuracy = 98.56%, Time = 38.67s
Task 9: Accuracy = 98.51%, Time = 38.07s
Task 10: Accuracy = 98.55%, Time = 38.96s

Best Agent Summary:
  Mean accuracy: 98.52% ± 0.05%
  Total time: 392.33s


### 🏆 YeWeN / Екатерина1

> **TorchMLP (RMSprop + FE + SiLU + Bottleneck + WeightNorm + LabelSmoothing + WarmupCosine + Dynamic INT8)**


The architecture expands from `256 → 128` with **Batch Normalization**, **SiLU + ReLU** activations, and a **Bottleneck residual block** to enhance gradient flow and feature retention.  
A **Weight Normalization** layer stabilizes training, while **Label Smoothing (ε=0.05)** improves robustness.

On the optimization side, **RMSprop** was selected as the most stable CPU-friendly optimizer, combined with a **Warmup-Cosine scheduler** for smoother convergence.  
During inference, **Dynamic INT8 quantization** compresses all linear layers, cutting memory usage and latency with minimal accuracy loss.

This design achieves **98.65% mean accuracy in only ~21 seconds** on the Permuted MNIST benchmark, demonstrating an ideal balance between computational efficiency and predictive precision.  
Overall, it is a **compact, high-performing, and deployable agent**, suitable for CPU-limited environments without sacrificing reliability.

---

**Final Configuration:**
| Component | Description |
|------------|--------------|
| Architecture | `MLP(256→128)` + BatchNorm + SiLU/ReLU + Bottleneck |
| Regularization | WeightNorm + Label Smoothing (ε=0.05) |
| Optimizer | RMSprop (lr=1e-3, α=0.99) |
| Scheduler | WarmupCosine (20 epochs, warmup=1) |
| Compression | Dynamic INT8 Quantization |
| Training | Epochs = 10, Batch = 128 |
| Mean Accuracy | **98.65%** |
| Total Runtime | **21.0 s** |
| Memory Usage | **~656 MB** |
| Agent Name (ML-Arena) | **YeWeN / Екатерина1** |


# Ⅳ Failure Analysis & Next Steps


## Failure Analysis

**1️⃣ Insufficient Utilization of Training Time**  
The final model completed all tasks in about **21 seconds**, leaving nearly 40 seconds unused out of the 60-second evaluation limit.  
This indicates that the computational budget was not fully exploited — more training epochs, structural depth, or small-scale ensembles could be implemented to extract higher accuracy within the allowed runtime.


**2️⃣ Excessive CPU Usage**  
During multi-task evaluation, CPU utilization occasionally exceeded 140% (equivalent to two cores running at full load with slight oversubscription), suggesting suboptimal thread management and data loading. Possible improvements include:  
- Reducing the number of `DataLoader` workers (`num_workers=0` or `1`) to minimize parallel overhead;  
- Adjusting the `batch_size` to balance compute and memory bandwidth;  
- Using `torch.set_num_threads(1)` before and after quantization to stabilize CPU usage.

---

## Improvement Directions and Future Work

**1️⃣ Increase Epochs + Early Stopping Mechanism**  
Since the model did not reach the time limit at 20 epochs, the training duration can be safely extended to **30 epochs or more**, combined with early stopping (e.g., `patience=3–5`) to prevent overfitting.  
This would allow more thorough convergence without wasting resources, leading to smoother validation curves and more stable accuracy.


**2️⃣ Dropout Regularization**  
Introducing moderate **Dropout (p≈0.1–0.2)** in the bottleneck or hidden layers can prevent neuron co-adaptation and improve robustness against noise and permutation variations.  
In a CPU-only setting, this lightweight regularization adds almost no computational cost while noticeably enhancing generalization and stability.


**3️⃣ Ensemble Learning**  
Training multiple lightweight models (using different random seeds or subsets of data) within the available time and averaging their predictions during inference can improve generalization.  
Even under CPU constraints, such ensembling can provide meaningful gains by reducing variance and smoothing out random errors at low additional cost.


**4️⃣ Meta-Learning**  
Future work can explore meta-learning approaches such as

